In [2]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

import spacy

In [3]:
tweetdata = pd.read_csv("../data/raw/testdata.manual.2009.06.14.csv", header=None, encoding="ISO-8859-1")
tweetdata.columns = ['polarity', 'id', 'date', 'source', 'username', 'text']

In [10]:
def get_max_length(docs):
    sentence_lengths = []
    for doc in docs:
        sentence_lengths.append(len(doc))
    return max(sentence_lengths)

In [11]:
get_max_length(tweetdata.text.apply(tokenizer.tokenize))

31

In [17]:
labels = tweetdata.polarity

In [4]:
# 1/21/19 https://github.com/hundredblocks/concrete_NLP_tutorial/blob/master/NLP_notebook.ipynb
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

In [6]:
tweetdata = standardize_text(tweetdata, 'text')
tweet_doc = tweetdata.text.tolist()

In [7]:
t = Tokenizer()
t.fit_on_texts(tweet_doc)

vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(tweet_doc)

In [9]:
sentence_lengths = []
for doc in encoded_docs:
    sentence_lengths.append(len(doc))
max_length = max(sentence_lengths)
print(max_length)

31


In [10]:
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [12]:
embedding_matrix = np.zeros((vocab_size, 300))


In [14]:
nlp = spacy.load('en_vectors_web_lg')

In [15]:
for word, i in t.word_index.items():
    embedding_vector = nlp.vocab.get_vector(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [26]:
# define model
model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=31, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(3, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 31, 300)           618600    
_________________________________________________________________
flatten_3 (Flatten)          (None, 9300)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 27903     
Total params: 646,503
Trainable params: 27,903
Non-trainable params: 618,600
_________________________________________________________________
None


In [28]:
from keras.utils import to_categorical

In [40]:
renum_labels = []
for label in labels:
    if label == 0:
        renum_labels.append(0)
    elif label == 2:
        renum_labels.append(1)
    else:
        renum_labels.append(2)

In [41]:
categorical_labels = to_categorical(renum_labels, num_classes=3)

In [42]:
model.fit(padded_docs, categorical_labels, epochs=5, verbose=0)

In [44]:
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, categorical_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 97.991968
